# COPT and LOLP Calculator

## import and settting printer

In [1]:
import itertools
import numpy as np
import pandas as pd

from tqdm.auto import tqdm

In [2]:
# setting numpy print option decimal places
decimal_places = 3
np.set_printoptions(precision=decimal_places, suppress=True)
pd.set_option('display.float_format', '{:.3f}'.format)

## inputs

### choose between using csv file or manually edit the notebook

In [3]:
# NOTE:
#   UNCOMMENT AND USE THIS CELL IF USING CSV INPUT
#   COMMENT IF USING MANUAL INPUT
# csv input path
df = pd.read_csv('../data/COPT Case.csv')

# extract csv
capacities = df['Capacity (MW)'].tolist()
outage_rates = df['FOR (Outage)'].tolist()
status = [True] * len(df.index)

In [4]:
# # NOTE:
# #   UNCOMMENT AND USE THIS CELL IF USING MANUAL INPUT
# #   COMMENT IF USING CSV INPUT
# # max capacity
# capacities = [
#     80,
#     70,
#     40,
#     50,
#     40,
#     50
# ]

# # forced outage rate
# outage_rates = [
#     0.1, 
#     0.2,
#     0.1,
#     0.2,
#     0.1,
#     0.1
# ]


# # status of generator,
# #     True: exist
# #     False: not exist
# status = [
#     True,
#     False,
#     True,
#     True,
#     True,
#     True,
# ]

In [5]:
pd.DataFrame(data={'capacities': capacities,
                   'outage_rates': outage_rates,
                   'status': status},
             index=pd.RangeIndex(1, len(capacities) + 1, 1)).head(10)

,capacities,outage_rates,status
1,50,0.100,True
2,50,0.100,True
3,50,0.100,True
4,30,0.100,True
5,10,0.070,True
6,20,0.070,True
7,10,0.100,True
8,5,0.100,True
9,10,0.070,True
10,10,0.070,True


In [20]:
# peak load or load at time considering
# used to calculate LOLP
demands = [590, 470, 585, 650, 660]

## COPT table

In [21]:
BATCH_SIZE = 1000
MIN_COVERAGE = 0.8
NUMBER_OF_CONSIDERED_GENERATOR = 20

In [22]:
ls = [[cap, out] for cap, out, stat in sorted(zip(capacities, outage_rates, status), reverse=True) if stat][:NUMBER_OF_CONSIDERED_GENERATOR]
capacities_fix = [[cap, 0] for cap, _ in ls]
probabilities_fix = [[1-out, out] for _, out in ls]

capacities_combination_generator = itertools.product(*capacities_fix)
probabilities_combinations_generator = itertools.product(*probabilities_fix)

capacity_merged = []
probability_merged = []

with tqdm(total=MIN_COVERAGE) as pbar:
    sum_probability = 0
    while sum_probability <= MIN_COVERAGE:
        capacity_merged += [sum(i) for i in itertools.islice(capacities_combination_generator, BATCH_SIZE)]
        probability_merged += [np.prod(i) for i in itertools.islice(probabilities_combinations_generator, BATCH_SIZE)]

        sum_probability = sum(probability_merged)

        pbar.update(n=sum_probability - pbar.n)

# combine capacity and individual probability
table = np.vstack([capacity_merged, probability_merged]).T

# sort the combined table
table = table[(-table[:, 0]).argsort(),:]

# combine duplicate
table = np.array([[k,sum([x[1] for x in list(g)])] 
                 for k,g in itertools.groupby(table, lambda x:x[0])])

# add cumulative probability
table = np.hstack((table, np.atleast_2d(np.cumsum(table[::-1,1])[::-1]).T))
table = np.hstack((table, np.atleast_2d(np.cumsum(table[:,1])).T))

  0%|          | 0/0.8 [00:00<?, ?it/s]

In [23]:
columns_name = ['Combined Capacity',
                'Individual Probability',
                'Cumulative Probability',
                'Reversed Cumulative Probability']

pd.DataFrame(data=table, columns=columns_name, index=pd.RangeIndex(1, len(table) + 1, 1))

,Combined Capacity,Individual Probability,Cumulative Probability,Reversed Cumulative Probability
1,670.000,0.188,0.801,0.188
2,660.000,0.021,0.613,0.209
3,650.000,0.137,0.592,0.346
4,640.000,0.036,0.455,0.382
5,630.000,0.046,0.419,0.428
6,620.000,0.128,0.373,0.556
7,610.000,0.022,0.245,0.578
8,600.000,0.084,0.223,0.662
9,590.000,0.022,0.138,0.685
10,580.000,0.028,0.116,0.712


## LOLP

In [24]:
def get_lolp(capacity, cumulative_probability, demand):
    """
    format:
        capacity (descend)
        cumulative_probability(descend)
    """
    try:
        idx = np.where(capacity < demand)[0][0]
    except IndexError:
        idx = -1
    return cumulative_probability[idx]

In [25]:
lolp = [get_lolp(table[:,0], table[:,2], demand) for demand in demands]
lolp

[0.11593100221055763,
 0.0004346090799862372,
 0.11593100221055763,
 0.4551052399888967,
 0.5916995114173529]

In [18]:
print(f'Total LOLP: {sum(lolp):.4f}')

Total LOLP: 3.8151
